In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
url = 'http://www.dot.ca.gov/mail.htm'
r = requests.get(url)
text = r.text

Steps to approach the problem: 
1. The data is available on website. Using requests and BeautifulSoup package we can get the data from web in the html format
2. The required data is in table, using BeautifulSoup we can find the table details of the page using find_all
3. For different website links , the links in "a href" have been appended to the base link 'http://www.dot.ca.gov'
4. Other details have been extracted using string manipulation function based on the format, by extracting details from td tags 

In [2]:
soup = BeautifulSoup(text, "html.parser")
soup_pretty = soup.prettify()

# Capturing links

In [3]:
url = 'http://www.dot.ca.gov'
office_link =[]
for trs in soup.find('table').find_all('tr'):
    for tds in trs.find_all('td'):
        if tds.find('a') and tds.find('a').get('href'):
            #print('***********')
            #print(tds)
            office_link.append(url + tds.a['href'])
          #  print(url + tds.a['href'])

## Capturing Other Details into Pandas DataFrame

In [4]:
office_name = []
mail_pobox =[]
mail_address =[]
mail_city=[]
mail_state = []
mail_zip = []
mail_phone=[]
office_address = []
office_city=[]
office_state=[]
office_zip = []
office_phone=[]
flag = 1
for trs in soup.find('table').find_all('tr')[:-1]:
    for tds in trs.find_all('td'):
            tmp = tds.text.split("\r\n")
            tmp = [x.strip(' ') for x in tmp]
            tmp = [x.replace(u'\xa0', u' ') for x in tmp]
            if flag==1:
                if len(tmp[0].split('\n'))==2:
                    office_address.append(tmp[0].split('\n')[1])
                    office_name.append(tmp[0].split('\n')[0])
                    if tmp[1].find(',')!=-1:
                        office_city.append(tmp[1].split(',')[0])
                        office_state.append(tmp[1].split(',')[1].strip().split(" ")[0])
                        office_zip.append(tmp[1].split(',')[1].strip().split(" ")[1])
                        office_phone.append(tmp[2])
                    else:
                        office_city.append(tmp[1])
                        office_state.append(np.NaN)
                        office_zip.append(np.NaN)
                        office_phone.append(tmp[2])
                    flag=2
                    continue
                    
                if len(tmp[1])<=2:
                    name = tmp[0]+" "+tmp[1]
                    office_address.append(tmp[2])
                    if tmp[3].find(',')!=-1:
                        office_city.append(tmp[3].split(',')[0])
                        office_state.append(tmp[3].split(',')[1].strip().split(" ")[0])
                        office_zip.append(tmp[3].split(',')[1].strip().split(" ")[1])
                        office_phone.append(tmp[4])
                    else:
                        office_city.append(tmp[3])  
                        office_state.append(np.NaN)
                        office_zip.append(np.NaN)
                        office_phone.append(tmp[4])
                else:
                    name = tmp[0]
                    office_address.append(tmp[1])
                    if tmp[2].find(",")!=-1:
                        office_city.append(tmp[2].split(',')[0])
                        office_state.append(tmp[2].split(',')[1].strip().split(" ")[0])
                        office_zip.append(tmp[2].split(',')[1].strip().split(" ")[1])
                        office_phone.append(tmp[3])
                    else:
                        office_city.append(tmp[2])
                        office_state.append(np.NaN)
                        office_zip.append(np.NaN)
                        office_phone.append(tmp[3])
                flag =2
                office_name.append(name)
            else:
                #Used regular expression to identify PO Box details
                if re.search( r'\s*[P|p]\s*[.]\s*[O|o]\s*[.]\s*[B/b][O|o][X|x]', tmp[0]):
                    mail_pobox.append(tmp[0])
                    mail_address.append(np.NaN)
                elif re.search( r'\s*[P|p]\s*[.]\s*[O|o]\s*[.]\s*[B/b][O|o][X|x]', tmp[1]):
                    mail_pobox.append(tmp[1])
                    mail_address.append(tmp[0])
                else:
                    mail_pobox.append(np.NaN)
                    mail_address.append(tmp[0])

                

                if tmp[1].find(",")!=-1:
                    mail_city.append(tmp[1].split(",")[0])
                    mail_state.append(tmp[1].split(",")[1].strip().split(" ")[0])
                    mail_zip.append(tmp[1].split(",")[1].strip().split()[1])
                    #Used regex to identify the contact number
                    if len(tmp)==3 and bool(re.match('[\d][\d|-]+$', tmp[2])):
                        mail_phone.append(tmp[2])
                    else:
                        mail_phone.append(np.NaN)
                elif tmp[2].find(",")!=-1:
                    mail_city.append(tmp[2].split(",")[0])
                    mail_state.append(tmp[2].split(",")[1].strip().split(" ")[0])
                    mail_zip.append(tmp[2].split(",")[1].strip().split()[1])
                    if len(tmp)==3 and bool(re.match('[\d][\d|-]+$', tmp[2])):
                        mail_phone.append(tmp[2])
                    else:
                        mail_phone.append(np.NaN)                  
                else:
                    mail_city.append(tmp[1])
                    mail_state.append(np.NaN)
                    mail_zip.append(np.NaN)
                    if len(tmp)==3 and bool(re.match('[\d][\d|-]+$', tmp[2])):
                        mail_phone.append(tmp[2])
                    else:
                        mail_phone.append(np.NaN)
                flag=1 
    
            
                    

                
            


In [5]:
import pandas as pd
data=pd.DataFrame(list(zip(office_name,office_link,office_city,office_address,office_state,office_zip,office_phone,
                      mail_address,mail_pobox,mail_city,mail_state,mail_zip,mail_phone)),
             columns=['office_name','office_link','office_city','office_address','office_state','office_zip','office_phone',
                      'mail_address','mail_pobox','mail_city','mail_state','mail_zip','mail_phone'])
             
             
             

In [6]:
data

,office_name,office_link,office_city,office_address,office_state,office_zip,office_phone,mail_address,mail_pobox,mail_city,mail_state,mail_zip,mail_phone
0,Headquarters,http://www.dot.ca.gov/,Sacramento,1120 N Street,NaN,NaN,916-654-5266,NaN,P.O. Box 942873,Sacramento,CA,94273-0001,NaN
1,District 1,http://www.dot.ca.gov/dist1/,Eureka,1656 Union Street,NaN,NaN,707-445-6600,NaN,P.O. Box 3700,Eureka,CA,95502,NaN
2,District 2,http://www.dot.ca.gov/dist2/,Redding,1657 Riverside Drive,NaN,NaN,530-225-3426,1657 Riverside Drive,NaN,Redding,CA,96001,NaN
3,District 3,http://www.dot.ca.gov/dist3/,Marysville,"703 ""B"" Street",NaN,NaN,530-741-4211,"703 ""B"" Street",NaN,Marysville,CA,95901,530-741-4211
4,District 4,http://www.dot.ca.gov/dist4/,Oakland,111 Grand Avenue,NaN,NaN,510-286-4444,111 Grand Ave,P.O. Box 23660,Oakland,CA,94623-0660,NaN
5,District 5,http://www.dot.ca.gov/dist05/,San Luis Obispo,50 Higuera Street,NaN,NaN,805-549-3111,50 Higuera Street,NaN,San Luis Obispo,CA,93401-5415,NaN
6,District 6,http://www.dot.ca.gov/dist6/,Fresno,1352 W. Olive Ave.,NaN,NaN,559-488-4020,NaN,P. O. Box 12616,Fresno,CA,93728-2616,NaN
7,District 7,http://www.dot.ca.gov/dist07/,Los Angeles,100 S. Main Street,NaN,NaN,213-897-3656,100 S. Main Street,NaN,Los Angeles,CA,90012,NaN
8,District 8,http://www.dot.ca.gov/dist8/,San Bernardino,464 W. 4th St.,NaN,NaN,909-383-4561,464 W. 4th St.,NaN,San Bernardino,CA,92402,NaN
9,District 9,http://www.dot.ca.gov/dist9/,Bishop,500 South Main Street,NaN,NaN,760-872-0601,500 South Main Street,NaN,Bishop,CA,93514,NaN


In [7]:
out_json=data.to_json(orient='records')
#out_json has the data in required json format


In [8]:
with open('out_json.txt', 'w') as f:
    f.write(out_json)